<a href="https://colab.research.google.com/github/GiovanniBorrelli/Software-Requirements-to-User-Stories/blob/main/LLAMAfinetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installazione dei pacchetti e avvio del server Ollama
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh
from IPython.display import clear_output

import os
import threading
import subprocess

def start_ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.start()
clear_output()

# Scarica il modello Llama 3.1:8b e installa lightrag
!ollama pull llama3.1:8b
!pip install -U lightrag[ollama]

from lightrag.core.trainer import Trainer
from lightrag.components.model_client import OllamaClient

pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏    0 B/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏ 1.9 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   0% ▕▏  20 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   1% ▕▏  58 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   2% ▕▏  84 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   2% ▕▏ 100 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   3% ▕▏ 132 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   4% ▕▏ 173 MB/4.7 GB                  pulling manifest 
pulling 8eeb52dfb3bb...   4% ▕▏ 201 MB/4.7 GB    

ModuleNotFoundError: No module named 'lightrag.core.trainer'

In [2]:
from lightrag.core.trainer import Trainer
from lightrag.components.model_client import OllamaClient

# Carica il dataset JSON per il fine-tuning
import json

with open("dataset.json", "r") as file:
    data = json.load(file)

# Prepara i dati di addestramento per il fine-tuning
train_data = [
    {
        "input": f"The system requirement is: {item['requirement']}",
        "output": item["user_story"]
    }
    for item in data
]

# Configura il client Ollama e le opzioni di addestramento
model_client = OllamaClient(model="llama3.1:8b")
trainer = Trainer(model_client=model_client)

# Esegui il fine-tuning
trainer.train(train_data, epochs=5, batch_size=2)

# Funzione per generare user stories da nuovi requisiti
def generate_user_story(requirement: str) -> str:
    prompt = f"Generate a user story based on the following requirement: {requirement}"
    response = model_client.generate(prompt)
    return response["text"]

# Esempio di utilizzo per generare una nuova user story
new_requirement = "The application should allow users to filter search results by category."
user_story = generate_user_story(new_requirement)
print("Generated User Story:", user_story)

ModuleNotFoundError: No module named 'lightrag.core.trainer'

In [3]:
!pip install transformers peft torch

In [ ]:
import json
import torch
from transformers import LlamaForCausalLM, LlamaTokenizer, Trainer, TrainingArguments
from peft import get_peft_model, LoraConfig, TaskType

# Carica il dataset JSON
with open("dataset.json", "r") as file:
    data = json.load(file)

# Prepara i dati in un formato testuale concatenato per il fine-tuning
train_texts = [
    f"Requirement: {item['requirement']}\nUser Story: {item['user_story']}"
    for item in data
]

# Modello e Tokenizer
model_name = "huggyllama/llama-7b"  # Sostituisci con il nome esatto del modello Llama
tokenizer = LlamaTokenizer.from_pretrained(model_name)
model = LlamaForCausalLM.from_pretrained(model_name)



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565 - if you l

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [1]:
# Configura il fine-tuning con LoRA per un adattamento efficiente
peft_config = LoraConfig(
    task_type=TaskType.CAUSAL_LM,
    r=8,  # gradi di libertà
    lora_alpha=16,
    lora_dropout=0.1
)
model = get_peft_model(model, peft_config)

# Tokenizza i dati
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)

# Prepara i dati per il Trainer
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings):
        self.encodings = encodings

    def __getitem__(self, idx):
        return {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}

    def __len__(self):
        return len(self.encodings["input_ids"])

train_dataset = Dataset(train_encodings)

# Configurazione degli argomenti di addestramento
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=1,
    save_steps=10,
    save_total_limit=2,
)

# Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
)



NameError: name 'LoraConfig' is not defined

In [ ]:

# Esegui il fine-tuning
trainer.train()

# Funzione per generare user stories da nuovi requisiti
def generate_user_story(requirement: str) -> str:
    input_text = f"Requirement: {requirement}\nUser Story:"
    inputs = tokenizer(input_text, return_tensors="pt")
    outputs = model.generate(**inputs, max_new_tokens=50)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Esempio di utilizzo per generare una nuova user story
new_requirement = "The application should allow users to filter search results by category."
user_story = generate_user_story(new_requirement)
print("Generated User Story:", user_story)

In [ ]:
!sudo apt-get install -y pciutils
!curl -fsSL https://ollama.com/install.sh | sh # download ollama api
from IPython.display import clear_output

# Create a Python script to start the Ollama API server in a separate thread

import os
import threading
import subprocess
import requests
import json

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

from IPython.display import clear_output
!ollama pull llama3.1:8b
clear_output()

!pip install -U lightrag[ollama]

from lightrag.core.generator import Generator
from lightrag.core.component import Component
from lightrag.core.model_client import ModelClient
from lightrag.components.model_client import OllamaClient, GroqAPIClient

import time

qa_template = r"""<SYS>
You are a helpful assistant.
</SYS>
User: {{input_str}}
You:"""

class SimpleQA(Component):
    def __init__(self, model_client: ModelClient, model_kwargs: dict):
        super().__init__()
        self.generator = Generator(
            model_client=model_client,
            model_kwargs=model_kwargs,
            template=qa_template,
        )

    def call(self, input: dict) -> str:
        return self.generator.call({"input_str": str(input)})

    async def acall(self, input: dict) -> str:
        return await self.generator.acall({"input_str": str(input)})

from lightrag.components.model_client import OllamaClient
from IPython.display import Markdown, display
model = {
    "model_client": OllamaClient(),
    "model_kwargs": {"model": "llama3.1:8b"}
}
qa = SimpleQA(**model)

In [ ]:
output=qa("""Make USER STORIES from the software requirements below.  these requirements designed to meet specific standards or guidelines in healthcare informatics, such as those from the HITSP (Health IT Standards Panel), IHE (Integrating the Healthcare Enterprise), and align with HIPAA (Health Insurance Portability and Accountability Act) regulations for protecting patient health information.
MAKE A LIST OF ALL USER STORIES! DONT JUST TELL ME THE KEY AREAS. DO IT, EVEN IF THE USER STORIES ARE MORE THAN 50!
Requirements:The system shall create and maintain a single patient record for each patient, storing key identifier information (e.g., system ID, medical record number) with the ability to store multiple identifiers.
The system shall provide the ability to merge duplicate patient records into a single patient record.
The system shall capture and maintain demographic information, including historic data (e.g., prior names, addresses), with the ability to modify and access this information as discrete data elements for patient care and reporting functions.
The system shall allow querying for a patient using multiple forms of identification.
The system shall capture and maintain the identity of all providers involved in a patient's care, including the principal provider.
The system shall maintain a coded list of problems or diagnoses, capturing details such as onset date, resolution date, chronicity, and any associated orders or medications, with the ability to update, display, and document changes.
The system shall capture and display free text comments associated with problems or diagnoses.
The system shall maintain and display a medication list for each patient, including prescribing details, medication history, and dates (ordering, start, modify, renewal, and end). The system shall support uncoded or free text medications and document changes.
The system shall provide the ability to manage a patient-specific medication list, including current medications, with functionality to exclude or print medications and document that a patient takes no medications.
The system shall capture and maintain an allergy and adverse reaction list, including user modifications, the type of reaction, and an indication of whether the list was reviewed. It shall also indicate if the patient has no known drug allergies or adverse reactions.
The system shall store, display, and manage comprehensive patient history, including medication and allergy information as discrete data elements.
The system shall capture, update, and display patient history, including structured, coded, and free text data, with the ability to record the presence and absence of conditions and capture history from external sources.
The system shall provide the ability to manage clinical documentation, including saving notes in progress, finalizing, cosigning, addending, and correcting notes. The system shall track and display the identity of users making changes and the date/time of those changes.
The system shall allow filtering, searching, and ordering of notes by provider or associated diagnosis, with the ability to enter free text notes and customize templates for structured data input.
The system shall capture and manage patient vital signs and other clinical data elements as discrete data, displaying measurements in both metric and English units, and indicating when values fall outside preset normal ranges. The system shall graph and calculate metrics like BMI over time.

 """)
display(Markdown(f"**Answer:** {output.data}"))

**Answer:** Here are the USER STORIES based on the software requirements:

**Patient Management**

1. As a healthcare provider, I want to create and maintain a single patient record for each patient, so that I can easily access and update patient information.
2. As a healthcare provider, I want the system to provide the ability to merge duplicate patient records into a single patient record, so that I can ensure accurate patient data.
3. As a healthcare provider, I want to capture and maintain demographic information for each patient, including historic data such as prior names and addresses, so that I can access this information for patient care and reporting functions.
4. As a healthcare provider, I want the system to allow querying for a patient using multiple forms of identification, so that I can quickly find patient records.

**Provider Management**

5. As a healthcare provider, I want to capture and maintain the identity of all providers involved in a patient's care, including the principal provider, so that I can track who has been caring for each patient.
6. As a healthcare provider, I want the system to display the list of providers associated with each patient record.

**Problem and Diagnosis Management**

7. As a healthcare provider, I want the system to maintain a coded list of problems or diagnoses for each patient, including details such as onset date, resolution date, chronicity, and any associated orders or medications.
8. As a healthcare provider, I want to capture and display free text comments associated with problems or diagnoses, so that I can document additional information about each condition.
9. As a healthcare provider, I want the system to update, display, and document changes to patient problems or diagnoses.

**Medication Management**

10. As a healthcare provider, I want the system to maintain and display a medication list for each patient, including prescribing details, medication history, and dates (ordering, start, modify, renewal, and end).
11. As a healthcare provider, I want the system to support uncoded or free text medications and document changes.
12. As a healthcare provider, I want the system to provide the ability to manage a patient-specific medication list, including current medications.

**Allergy and Adverse Reaction Management**

13. As a healthcare provider, I want the system to capture and maintain an allergy and adverse reaction list for each patient, including user modifications, the type of reaction, and an indication of whether the list was reviewed.
14. As a healthcare provider, I want the system to indicate if the patient has no known drug allergies or adverse reactions.

**Patient History Management**

15. As a healthcare provider, I want the system to store, display, and manage comprehensive patient history, including medication and allergy information as discrete data elements.
16. As a healthcare provider, I want the system to capture, update, and display patient history, including structured, coded, and free text data.

**Clinical Documentation Management**

17. As a healthcare provider, I want the system to provide the ability to manage clinical documentation for each patient, including saving notes in progress.
18. As a healthcare provider, I want the system to track and display the identity of users making changes and the date/time of those changes.
19. As a healthcare provider, I want the system to allow filtering, searching, and ordering of notes by provider or associated diagnosis.

**Vital Signs Management**

20. As a healthcare provider, I want the system to capture and manage patient vital signs and other clinical data elements as discrete data, displaying measurements in both metric and English units.
21. As a healthcare provider, I want the system to indicate when values fall outside preset normal ranges.
22. As a healthcare provider, I want the system to graph and calculate metrics like BMI over time.

**Reporting and Analytics**

23. As a healthcare provider, I want the system to provide reports on patient data, including medication lists, allergy lists, and vital signs.
24. As a healthcare provider, I want the system to support data export for external analysis and reporting.
25. As a healthcare provider, I want the system to generate alerts and notifications based on patient data and clinical guidelines.

**Security and Compliance**

26. As a healthcare provider, I want the system to ensure the confidentiality, integrity, and availability of patient health information, in accordance with HIPAA regulations.
27. As a healthcare provider, I want the system to implement access controls, including authentication and authorization, to restrict access to authorized users only.

**User Interface**

28. As a healthcare provider, I want the system to provide an intuitive user interface that is easy to navigate and use.
29. As a healthcare provider, I want the system to support multiple devices and platforms, including desktop computers, laptops, tablets, and smartphones.

Note: These USER STORIES are based on the provided requirements and may need to be refined or modified as needed to ensure they accurately capture the user's needs and behaviors.